In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_metric

from sklearn.model_selection import StratifiedKFold


In [10]:
data = DatasetDict({
    'train': load_dataset('csv', data_files=['./data/shai-training-2024-a-level-2/Train.csv','./data/shai-training-2024-a-level-2/Valid.csv'], split='train'),
    'test':load_dataset('csv', data_files='./data/shai-training-2024-a-level-2/Test.csv', split='train')
})

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

checkpoint = 'JamesH/Movie_review_sentiment_analysis_model'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, truncation=True, cache_dir='./models_weights')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
def preprocess_data(batch):
    return tokenizer(batch['text'])

data = data.map(preprocess_data, batched=True)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
})

# TrainerAPI

In [5]:
def compute_metrics(eval_pred):
    metric = load_metric('accuracy')
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=labels)



def train_model(data, cv_idx):

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2, cache_dir='./models_weights')

    train_args = TrainingArguments(
        output_dir=f'results_debertav2_cv/split{cv_idx}/',
        logging_dir='logs',
        report_to=[],
        
        evaluation_strategy='steps',
        eval_steps=5000,
        logging_steps=5000,
        save_steps=5000,
        per_device_eval_batch_size=1,
        per_device_train_batch_size=1,
        learning_rate=1e-6,
        weight_decay=0.005,
        num_train_epochs=5,
        fp16=True,
        
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        save_total_limit=3,
    )
    trainer = Trainer(
        model=model,
        args=train_args,
        data_collator=data_collator,
        train_dataset=data['train'],
        eval_dataset=data['val'],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(3)]
    )
    trainer.train()



In [7]:
folds = StratifiedKFold(n_splits=5)

splits = folds.split(np.zeros(data["train"].num_rows), data["train"]["label"])


for i, (train_idxs, val_idxs) in enumerate(splits):
    fold_dataset = DatasetDict({
    "train":data["train"].select(train_idxs),
    "val":data["train"].select(val_idxs),
    "test":data["test"]
    })
    train_model(fold_dataset, i)

  0%|          | 0/180000 [00:00<?, ?it/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.2947, 'learning_rate': 9.72272222222222e-07, 'epoch': 0.14}


  0%|          | 0/9000 [00:00<?, ?it/s]

/tmp/ipykernel_59801/3468046051.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


...

{'eval_loss': 0.2542932331562042, 'eval_accuracy': 0.9624444444444444, 'eval_runtime': 56.4297, 'eval_samples_per_second': 159.49, 'eval_steps_per_second': 159.49, 'epoch': 1.25}
{'loss': 0.2122, 'learning_rate': 7.223666666666667e-07, 'epoch': 1.39}


  0%|          | 0/9000 [00:00<?, ?it/s]

{'eval_loss': 0.2961835265159607, 'eval_accuracy': 0.9606666666666667, 'eval_runtime': 56.5032, 'eval_samples_per_second': 159.283, 'eval_steps_per_second': 159.283, 'epoch': 1.39}
{'loss': 0.2104, 'learning_rate': 6.946e-07, 'epoch': 1.53}


  0%|          | 0/9000 [00:00<?, ?it/s]

{'eval_loss': 0.25824299454689026, 'eval_accuracy': 0.9625555555555556, 'eval_runtime': 56.1692, 'eval_samples_per_second': 160.23, 'eval_steps_per_second': 160.23, 'epoch': 1.53}
{'train_runtime': 2978.6988, 'train_samples_per_second': 60.429, 'train_steps_per_second': 60.429, 'train_loss': 0.2561256613991477, 'epoch': 1.53}


In [6]:
model1 = AutoModelForSequenceClassification.from_pretrained('./results_debertav2_cv/split0/checkpoint-30000/', num_labels=2)
model2 = AutoModelForSequenceClassification.from_pretrained('./results_debertav2_cv/split1/checkpoint-60000', num_labels=2)
model3 = AutoModelForSequenceClassification.from_pretrained('./results_debertav2_cv/split2/checkpoint-80000', num_labels=2)
model4 = AutoModelForSequenceClassification.from_pretrained('./results_debertav2_cv/split3/checkpoint-85000', num_labels=2)
model5 = AutoModelForSequenceClassification.from_pretrained('./results_debertav2_cv/split4/checkpoint-40000', num_labels=2)

In [12]:
train_args = TrainingArguments(
    output_dir=f'results_debertav2_cv/test/',
    logging_dir='logs',
    report_to=[],
    
    evaluation_strategy='steps',
    eval_steps=5000,
    logging_steps=5000,
    save_steps=5000,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=1,
    learning_rate=1e-6,
    weight_decay=0.005,
    num_train_epochs=5,
    fp16=True,
    
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    save_total_limit=3,
)
#data['test'] = data['test'].remove_columns(['label'])
data['train'] = data['train'].remove_columns(['label'])
    

In [15]:
dataset_split = 'train'

In [14]:
import pickle

In [16]:
trainer = Trainer(
    model=model1,
    args=train_args,
    data_collator=data_collator,
    train_dataset=data['train'],
    #eval_dataset=data['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(3)]
)

preds = trainer.predict(data[dataset_split])

with open('predictions/cv_train/debertav2_split0.pkl', 'wb') as f:
    pickle.dump(preds.predictions, f)

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/45000 [00:00<?, ?it/s]

In [17]:
trainer = Trainer(
    model=model2,
    args=train_args,
    data_collator=data_collator,
    train_dataset=data['train'],
    #eval_dataset=data['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(3)]
)

preds = trainer.predict(data[dataset_split])

with open('predictions/cv_train/debertav2_split1.pkl', 'wb') as f:
    pickle.dump(preds.predictions, f)

  0%|          | 0/45000 [00:00<?, ?it/s]

In [18]:
trainer = Trainer(
    model=model3,
    args=train_args,
    data_collator=data_collator,
    train_dataset=data['train'],
    #eval_dataset=data['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(3)]
)

preds = trainer.predict(data[dataset_split])

with open('predictions/cv_train/debertav2_split2.pkl', 'wb') as f:
    pickle.dump(preds.predictions, f)

  0%|          | 0/45000 [00:00<?, ?it/s]

In [19]:
trainer = Trainer(
    model=model4,
    args=train_args,
    data_collator=data_collator,
    train_dataset=data['train'],
    #eval_dataset=data['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(3)]
)

preds = trainer.predict(data[dataset_split])

with open('predictions/cv_train/debertav2_split3.pkl', 'wb') as f:
    pickle.dump(preds.predictions, f)

  0%|          | 0/45000 [00:00<?, ?it/s]

In [20]:
trainer = Trainer(
    model=model5,
    args=train_args,
    data_collator=data_collator,
    train_dataset=data['train'],
    #eval_dataset=data['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(3)]
)

preds = trainer.predict(data[dataset_split])

with open('predictions/cv_train/debertav2_split4.pkl', 'wb') as f:
    pickle.dump(preds.predictions, f)

  0%|          | 0/45000 [00:00<?, ?it/s]